# Pandas for time series

Pandas is very useful for handling time series. 

First we'll need some data. I started at [Diskos](https://portal.diskos.cgg.com/whereoil-data/). It's a bit confusing as there are a lot of places to get data, but I've heard of FactPages so let's start there.

## FactPages... Use pandas to read CSV directly

Right-click and copy URL for CSV from this link:

http://factpages.npd.no/factpages/Default.aspx?culture=nb-no&nav1=field&nav2=TableView|Production|Saleable|Monthly

This file is saved in `../data/field_production_monthly.csv` as well, in case the link breaks.

In [ ]:
csv = "https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/field_production_monthly&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=CSV&Top100=false&IpAddress=165.225.35.29&CultureCode=nb-no"

In [ ]:
import pandas as pd

df = pd.read_csv(csv)

In [ ]:
df.head()

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
<h3>Exercise</h3>

- How many rows are there in this dataframe?
- How many fields are represented? (Look at the column called `'prfInformationCarrier'`)
- How many years of data are there?
- What is the total production?
</div>

## Rename some columns

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
<h3>Exercise</h3>

Rename some of the columns of the dataframe as follows:

    'prfYear': 'year'
    'prfMonth': 'month'
    'prfInformationCarrier': 'field'
    'prfPrdOilNetMillSm3': 'oil'
    'prfPrdOeNetMillSm3': 'OE'
    'prfPrdProducedWaterInFieldMillSm3': 'water'
</div>

## Add a datetime

We'd like to give this dataframe a **datetime** index with `pandas` datetimes. To do this easily, we need:

- EITHER columns named like `'year'`, `'month'`, `'day'`
- OR a column with a datetime string like `2019-06-30`.

In this dataframe, we have the former, so let's work with that.

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
<h3>Exercise</h3>

- Make a column for the **day**, using a constant like 1.
- Make a datetime column called `'ds'` (for 'date stamp') using `pd.to_datetime()`, passing in a dataframe consisting of the three columns for year, month and the day you just made.
- Finally, to turn the new column into an index, give its name (`'ds'`) to `df.set_index()`.
</div>

You should end up with a new dataframe with the `'ds'` column as an index.

In [ ]:
df.head()

## Simplify the dataframe

Before we carry on, let's simplify the dataframe a bit, reducing it to a few columns: **field**, **water**, **other**, and **oil** (the order is a slightly cheaty way to get the colours I want on the charts, without having to fiddle with them).

In [ ]:
df['other'] = df.OE - df.oil
df = df.drop('OE', axis=1)
df = df[['field', 'water', 'other', 'oil']]

In [ ]:
df.head()

## Time series with `pandas`

`pandas` knows all about time series. So we can easily make a time series plot:

In [ ]:
df.oil[df.field=='TROLL'].plot()

We can easily stretch it out, or add other lines:

In [ ]:
df[df.field=='TROLL'].plot(figsize=(15,3))

Let's make a dataframe of only the TROLL field.

In [ ]:
troll = df[df.field=='TROLL']

Now we can slice using natural dates:

In [ ]:
troll['2005':'2010'].plot()

In [ ]:
troll['Jun 2005':'Jun 2007'].plot()

Try to imagine doing that in Excel!

Let's get the summed annual production for the Troll field:

In [ ]:
troll.loc['2010':'2018'].resample('Y').sum()

Throw `.plot()` on the end:

In [ ]:
troll.loc['1995':'2018'].resample('Y').sum().plot()

Or we can get totals for *ALL* fields in the database:

In [ ]:
df.loc['2000':'2018'].resample('Y').sum().plot()

Let's look at the contribution TROLL made to NCS production since 1993:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
df.loc['1993':'2018', 'oil'].resample('Y').sum().plot(ax=ax)
df.loc[df.field!='TROLL'].loc['1993':'2018', 'oil'].resample('Y').sum().plot(ax=ax)
plt.show()

Looking for forecasting? Head over to...

### [Time series forecasting](Time_series_forecasting.ipynb.ipynb)

## Add operators

There's also a list of operators here >> https://factpages.npd.no/factpages/Default.aspx?culture=nb-no&nav1=field&nav2=TableView%7cProduction%7cSaleable%7cMonthly

This file is also in `/data`, but we can read directly from the web with `pandas`, as before:

In [ ]:
url = "https://factpages.npd.no/ReportServer_npdpublic?/FactPages/TableView/field_operator_hst&rs:Command=Render&rc:Toolbar=false&rc:Parameters=f&rs:Format=CSV&Top100=false&IpAddress=165.225.35.29&CultureCode=nb-no"
dg = pd.read_csv(url)

In [ ]:
dg.head()

In [ ]:
dg['from'] = pd.to_datetime(dg.fldOperatorFrom)
dg['to'] = pd.to_datetime(dg.fldOperatorTo)
dg['to'] = dg['to'].fillna(pd.to_datetime('today'))

In [ ]:
dg.head()

Let's get the operator of each field, for each month, and put it in our production dataframe.

There is probably a more elegant way to do this with `join` or `merge` or something... but I can't figure it out.

In [ ]:
def process_row(row):
    """
    Process a row in df to get the operator at that time.
    
    Note that 'name' is a special attribute for the current index.
    """
    this_df = dg.loc[dg.fldName==row.field, :]
    record = this_df.loc[(dg['from'] < row.name) & (row.name <= dg['to']), "cmpLongName"]
    if any(record.values):
        return record.values[0]
    else:
        return np.nan

In [ ]:
df['operator'] = df.apply(process_row, axis=1)

In [ ]:
plt.figure(figsize=(10,6))
for name, group in df[df.field=='TROLL'].groupby('operator'):
    plt.plot(group.oil, label=name)
plt.legend()

----

(c) Agile Scientific 2019, licensed CC-BY